# PA1.1 Text Generation using Shannon Visualization Method

### Introduction

In this notebook, you will be generating text using the Shannon Visualization method.

An n-gram is a contiguous sequence of n words. For example "Machine" is a unigram, "Machine Learning" is a bigram and "Machine Learning PA1" is a trigram. In language modeling, n-gram models are probabilistic models of text that use word dependencies and context to predict the likelihood of occurence of an n-gram, i.e. predicting the nth word in an n-gram based on the previous n-1 words. One use of the predictions made by such a model is text generation. In this part, you will be generating text using the Shannon Visualization Method.

For additional details of the working of n-gram models and shannon visualization method, you can also consult [Chapter 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) of the SLP3 book as reference.

### Instructions

- Follow along with the notebook, filling out the necessary code where instructed.

- <span style="color: red;">Read the Submission Instructions, Plagiarism Policy, and Late Days Policy in the attached PDF.</span>

- <span style="color: red;">Make sure to run all cells for credit.</span>

- <span style="color: red;">Do not remove any pre-written code.</span>

- <span style="color: red;">You must attempt all parts.</span>

For this notebook, in addition to standard libraries i.e. `numpy`, `pandas`, `regex`, `matplotlib` and `scipy`, you **can** use [UrduHack](https://github.com/urduhack/urduhack) for tokenization, and [NLTK](https://www.nltk.org/) for training your n-grams. However, no other machine learning toolkits or libraries are allowed.

In [ ]:
# import all required libraries here
import numpy as np
import pandas as pd
import os
import string

import nltk
nltk.download('punkt')
!pip install urduhack
from urduhack.tokenization import sentence_tokenizer, word_tokenizer
from urduhack.normalization import normalize_characters
from urduhack.normalization import remove_diacritics
from urduhack import normalize
from nltk.tokenize import word_tokenize
from urduhack.tokenization import sentence_tokenizer
from urduhack.preprocessing import normalize_whitespace, remove_punctuation, remove_accents
from nltk import bigrams


import regex as re
from collections import Counter
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Dataset

You will be using the Urdu short stories by Patras Bukhari given in the folder `Urdu Short Stories` in the attached zip file for the purposes of this part of the assignment. This contains 6 stories of varying lengths which will serve as inputs for your n-gram model.

You're required to implement an n-gram model that uses the given stories to generate Urdu text that mimics the input stories.

## Loading and Preprocessing the Dataset

Read in the short story files given and tokenize the text to be preprocessed.

In [ ]:
# code here

# Doing everything together in the next cell

Preprocess the tokenized data. Go through the data and use your own discretion to decide on what kind of pre-processing might be required.

In [ ]:
# code here
def preprocess_text(data_folder):
  short_stories = list()
  for file_name in os.listdir(data_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(data_folder, file_name)
        with open(file_path, "r") as file:
            story = file.read()
            story = remove_punctuation(story)
            story = normalize(story)
            story = remove_accents(story)
            short_stories.append(story)
  return short_stories

# code here
data_folder = fr"/content/drive/MyDrive/GEN-AI/PA1/DataP1"
pre_processed_stories = preprocess_text(data_folder)
num_stories = len(pre_processed_stories)
print(f"Number of short stories loaded: {num_stories}")

tokenized_words = [word_tokenize(story) for story in pre_processed_stories]
tokenized_sentence = [sentence_tokenizer(story) for story in pre_processed_stories]
# print(tokenized_sentence)
print("Preprocessed tokens of the first story:")
print(tokenized_words[0])
print("Total stories tokenized:",len(tokenized_words))
print("Tokens in each story:")
k=1
for i in tokenized_words:
    print(f"story_{k}",len(i))
    k+=1

Number of short stories loaded: 6
Preprocessed tokens of the first story:
['سینما', 'کا', 'عشق', 'عنوان', 'تو', 'عجب', 'ہوس', 'خیز', 'ہے', 'لیکن', 'افسوس', 'کہ', 'اس', 'مضمون', 'سے', 'آپ', 'کی', 'تمام', 'توقعات', 'مجروح', 'ہوں', 'گی', 'کیونکہ', 'مجھے', 'تو', 'اس', 'مضمون', 'میں', 'کچھ', 'دل', 'کے', 'داغ', 'دکھانے', 'مقصود', 'ہیں', 'اس', 'سے', 'آپ', 'یہ', 'نہ', 'سمجھئے', 'کہ', 'مجھے', 'فلموں', 'سے', 'دلچسپی', 'نہیں', 'یا', 'سینما', 'کی', 'موسیقی', 'اور', 'تاریکی', 'میں', 'جو', 'ارمان', 'انگیزی', 'ہے', 'میں', 'اس', 'کا', 'قائل', 'نہیں', 'میں', 'تو', 'سینما', 'کے', 'معاملے', 'میں', 'اوائل', 'عمر', 'ہی', 'سے', 'بزرگوں', 'کا', 'مورد', 'عتاب', 'رہ', 'چکا', 'ہوں', 'لیکن', 'آج', 'کل', 'ہمارے', 'دوست', 'مرزا', 'صاحب', 'کی', 'مہربانیوں', 'کی', 'طفیل', 'سینما', 'گویا', 'میری', 'ایک', 'دکھتی', 'رگ', 'بن', 'کر', 'رہ', 'گیا', 'ہے', 'جہاں', 'اس', 'کا', 'نام', 'سن', 'پاتا', 'ہوں', 'بعض', 'دردانگیز', 'واقعات', 'کی', 'یاد', 'تازہ', 'ہوجاتی', 'ہے', 'جس', 'سے', 'رفتہ', 'رفتہ', 'میری', 'فطرت', 'ہی', 'کج', 

## Creating Unigrams

Generate a list of unigrams. Print the first 10 unigrams obtained.

In [ ]:
# code here
# my_corpus_unigram = [element for sublist in tokenized_words for element in sublist]
# print(len(my_corpus_unigram))
# print("first 10 unigrams:", my_corpus_unigram[:10])

# Adding <s> and <\s> tags in the sentences
my_corpus = [element for sublist in tokenized_sentence for element in sublist]
print("Total sentences in all stories:",len(my_corpus))
print("first 10 Sentences:", my_corpus[:10])
tokenized_words = list()
for story in my_corpus:
    tokenized_words.append(["<s>"] + word_tokenize(story) + ["<\s>"])


my_corpus = [element for sublist in tokenized_words for element in sublist]
print("My corpus:",my_corpus)
print("first 10 unigrams:", my_corpus[:10])


Total sentences in all stories: 587
first 10 Sentences: ['سینما کا عشق عنوان تو عجب ہوس خیز ہے لیکن افسوس کہ اس مضمون سے آپ کی تمام توقعات مجروح ہوں گی', 'کیونکہ مجھے تو اس مضمون میں کچھ دل کے داغ دکھانے مقصود ہیں', 'اس سے آپ یہ نہ سمجھئے کہ مجھے فلموں سے دلچسپی نہیں یا سینما کی موسیقی اور تاریکی میں جو ارمان انگیزی ہے', 'میں اس کا قائل نہیں میں تو سینما کے معاملے میں اوائل عمر ہی سے بزرگوں کا مورد عتاب رہ چکا ہوں لیکن آج کل ہمارے دوست مرزا صاحب کی مہربانیوں کی طفیل سینما گویا میری ایک دکھتی رگ بن کر رہ گیا ہے', 'جہاں اس کا نام سن پاتا ہوں', 'بعض دردانگیز واقعات کی یاد تازہ ہوجاتی ہے جس سے رفتہ رفتہ میری فطرت ہی کج بیں بن گئی ہے', 'اول تو خدا کے فضل سے ہم سینما کبھی وقت پر نہیں پہنچ سکے اس میں میری سستی کو ذرا دخل نہیں یہ سب قصور ہمارے دوست مرزا صاحب کا ہے جو کہنے کو تو ہمارے دوست ہیں لیکن خدا شاہد ہے', 'ان کی دوستی سے جو نقصان ہمیں پہنچے ہیں', 'کسی دشمن کے قبضئہ قدرت سے بھی باہر ہوں گے', 'جب سینما جانے کا ارادہ ہو ہفتہ بھر پہلے سے انہیں کہہ رکھتا ہوں کہ کیوں بھئی مرزا اگلی جمعرات سینم

Find the probabilities for each unique unigram. (Refer to the Shannon Visualization Method that we studied in class.)

In [ ]:
# code here

unigram_counts = Counter(my_corpus)
print(unigram_counts)
total_unigrams = len(my_corpus)
unigram_probabilities = {unigram: count / total_unigrams for unigram, count in unigram_counts.items()}
print("Probabilities for the first 10 unique unigrams:")
for unigram, probability in list(unigram_probabilities.items())[:10]:
    print(f"{unigram}: {probability}")

Counter({'<s>': 587, '<\\s>': 587, 'میں': 494, 'کے': 452, 'ہے': 361, 'اور': 352, 'کی': 340, 'سے': 307, 'کہ': 263, 'اس': 245, 'کا': 243, 'تو': 220, 'کو': 217, 'ہیں': 200, 'پر': 162, 'ایک': 156, 'نے': 152, 'یہ': 148, 'کر': 144, 'بھی': 125, 'ہوں': 117, 'نہ': 117, 'ہو': 113, 'نہیں': 110, 'ان': 109, 'آپ': 108, 'ہم': 108, 'لیکن': 100, 'کیا': 95, 'ہی': 84, 'وہ': 82, 'تھا': 82, 'جو': 69, 'پھر': 63, 'کوئی': 58, 'یا': 57, 'اب': 54, 'ہوا': 48, 'کسی': 47, 'بعد': 47, 'ہر': 47, 'تھے': 47, 'رہے': 44, 'لاہور': 44, 'تک': 43, 'کچھ': 42, 'بہت': 42, 'لیے': 42, 'اپنے': 42, 'گے': 41, 'وقت': 40, 'جب': 39, 'گیا': 38, 'دیا': 38, 'ساتھ': 37, 'اپنی': 36, 'کہا': 34, 'صرف': 34, 'دو': 34, 'کرتے': 34, 'طرح': 34, 'لئے': 34, 'سال': 34, 'سب': 33, 'دل': 32, 'جس': 32, 'بجے': 32, 'ہاسٹل': 32, 'مجھے': 31, 'ہوتا': 31, 'گا': 30, 'پاس': 30, 'ہمارے': 29, 'شروع': 29, 'جاتا': 29, 'جائے': 29, 'صاحب': 28, 'ہمیں': 28, 'رہا': 28, 'ضرور': 28, 'میری': 27, 'بعض': 27, 'ہونے': 27, 'تھی': 27, 'گئی': 26, 'دن': 26, 'پہلے': 25, 'بات': 25, 'ہ

## Creating Bigrams

Generate a list of bigrams. Print the first 10 bigrams obtained.

In [ ]:
# def n_grams_pp(tokenized_words):
#     for i in tokenized_words:
#         i.insert(0, "<s>")
#         for j in i:
#             if "؟" in j:
#                 dummy_index = int(i.index(j))
#                 i[dummy_index] = j.replace("؟","")
#                 i.insert(dummy_index+1,"</s>")
#                 i.insert(dummy_index+2,"<s>")
#             elif "۔" in j:
#                 dummy_index = int(i.index(j))
#                 i[dummy_index] = j.replace("۔","")
#                 i.insert(dummy_index+1,"</s>")
#                 i.insert(dummy_index+2,"<s>")
#         if i[-1]=="<s>":
#             i.pop()
#         print("hui hui",i)
#     return tokenized_words


In [ ]:
bigrams = [(my_corpus[i], my_corpus[i + 1]) for i in range(len(my_corpus) - 1)]

print("First 10 bigrams:")
print(bigrams[:10])

First 10 bigrams:
[('<s>', 'سینما'), ('سینما', 'کا'), ('کا', 'عشق'), ('عشق', 'عنوان'), ('عنوان', 'تو'), ('تو', 'عجب'), ('عجب', 'ہوس'), ('ہوس', 'خیز'), ('خیز', 'ہے'), ('ہے', 'لیکن')]


Find the probabilities for each unique bigram.

In [ ]:
# code here
bigram_counts = Counter(bigrams)  # contains tuple count
bigram_counts_individual = Counter(my_corpus)  # Contains unigram count but wit <s></s>
print(bigram_counts)
print(bigram_counts_individual)
total_bigrams = len(bigrams)
print(bigram_counts)
bigram_probabilities = dict()
for i in bigram_counts:
    # print(f"N--> {bigram_counts[i]}")
    # print(f"D-->{bigram_counts_individual[i[0]]}")
    bigram_probabilities[i] = bigram_counts[i]/bigram_counts_individual[i[0]]

# bigram_probabilities = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}
print("First ten Bigram Probabilities:")
itr = 0
for bigram, probability in bigram_probabilities.items():
    print(f"{bigram}: {probability:.15f}")
    if itr == 10:
        break
    itr+=1


Counter({('<\\s>', '<s>'): 586, ('ہے', '<\\s>'): 221, ('ہیں', '<\\s>'): 133, ('ہوں', '<\\s>'): 62, ('تھا', '<\\s>'): 50, ('<s>', 'اس'): 49, ('ہے', 'کہ'): 46, ('کے', 'بعد'): 41, ('میں', 'نے'): 40, ('ہم', 'نے'): 35, ('ہے', 'اور'): 35, ('اس', 'کے'): 34, ('تھے', '<\\s>'): 33, ('ان', 'کے'): 30, ('ہیں', 'اور'): 27, ('کے', 'لیے'): 26, ('آپ', 'کو'): 26, ('ہے', 'لیکن'): 25, ('کے', 'لئے'): 24, ('<s>', 'میں'): 23, ('اس', 'سے'): 22, ('گے', '<\\s>'): 22, ('<s>', 'یہ'): 22, ('میں', 'سے'): 21, ('ان', 'کی'): 20, ('تھا', 'کہ'): 20, ('<s>', 'وہ'): 20, ('گا', '<\\s>'): 20, ('تھی', '<\\s>'): 20, ('لالہ', 'جی'): 20, ('کے', 'ساتھ'): 19, ('جاتے', 'ہیں'): 18, ('<s>', 'ان'): 17, ('اس', 'کی'): 17, ('ہاسٹل', 'میں'): 17, ('نے', 'کہا'): 17, ('اس', 'میں'): 16, ('ہوتا', 'ہے'): 16, ('کے', 'متعلق'): 16, ('ہوں', 'اور'): 15, ('جاتا', 'ہے'): 15, ('اس', 'کا'): 14, ('ہوں', 'کہ'): 14, ('<s>', 'اب'): 14, ('اگلے', 'سال'): 14, ('میں', 'اس'): 13, ('اور', 'پھر'): 13, ('ہوتے', 'ہیں'): 13, ('اور', 'اس'): 13, ('اس', 'پر'): 13, ('می

## Generating Text using the Shannon Visualization Method

Generate a paragraph with ten sentences. Use the Shannon visualization method that we studied in class.

In [ ]:
# code here

print("type of unigram",type(unigram_probabilities))
print("type of bigram",type(bigram_probabilities))
def get_keys_by_value(dictionary, target_value):
    return [key for key, value in dictionary.items() if value <= target_value]

def get_dict_with_next_word_prob(tup, dic):
    all_possible_next_words_dict = {key: value for key, value in dic.items() if key[0] == tup[1]}
    max_value = max(all_possible_next_words_dict.values())  # max prob value
    # return {key: value for key, value in all_possible_next_words_dict.items() if value == max_value}
    return all_possible_next_words_dict

def generate_sentence(starting_word, probabilities, num_words=5):
    if isinstance(starting_word, tuple):
        sentence = list(starting_word)  # ("<s>", "some_word")
    else:
        sentence = [starting_word]
    # print("sentence before:",sentence)
    # return
    next_word = str()
    for _ in range(num_words - 1):
        if next_word == "<\s>":
            break

        if isinstance(starting_word, tuple):
            # getting all probs where starting is x.
            next_word_probability_dict = get_dict_with_next_word_prob(starting_word, probabilities)
            # print("next_word_probability_dict",next_word_probability_dict)
            next_word_tup = random.choices(
                list(next_word_probability_dict.keys()),
                weights=list(next_word_probability_dict.values())
            )[0]
            next_word = next_word_tup[1]  # extract the next word from the tuple
            starting_word = next_word_tup # Now, the starting tuple will be the current tuple
            # print("next_word",next_word)
        else:
            next_word = random.choices(
                list(probabilities.keys()),
                weights=list(probabilities.values())
            )[0]
            # print("hui",next_word)
        if next_word == "<s>":
            continue
        sentence.append(next_word)
    # print("sentence after:",sentence)
    return ' '.join(["<s>"]+sentence[1:-1]+["<\s>"])

def generate_paragraph(starting_words, probabilities, num_sentences=10, num_words_per_sentence=5, use_unigram="True"):
    paragraph = list()
    for i in range(num_sentences):
        if use_unigram:
            starting_word = "<s>"
            # break
        else:
            starting_word = ("<s>",random.choice(starting_words[1:]))
        sentence = generate_sentence(starting_word, probabilities, num_words_per_sentence)
        paragraph.append(sentence)
        print(fr"Sentence No.{i+1}: {sentence}")

    return paragraph



# starting_string = "لاہور کا شہر وہ شہر جہاں ہر گلی میں ایک خوش نویس رہتا ہے اور جہاں حکیم فقیر محمد مرحوم جیسے استاد فن پیدا ہوئے"
# starting_string = "ٓنے والی نسلوں کی بہبودی کا انحصار ہے"
starting_string = "ٓتھرڈ ڈویزن میں پاس ہونے کی وجہ سے یونیورسٹی نے ہم کو وظیفہ دینا مناسب نہ سمجھا"
starting_string = normalize_whitespace(starting_string)
starting_string = normalize_characters(starting_string)   # Keeping everything in Urdu unicode
starting_string = remove_accents(starting_string)
starting_string = remove_punctuation(starting_string)
starting_words_u =  word_tokenize(starting_string)
starting_words_b =  word_tokenize(starting_string)
print("starting_words_b:",starting_words_b)

no_of_sentences = 10
no_of_words_each_sentence = 20

print("\n\n\nUNIGRAM")
paragraph_uni = generate_paragraph(starting_words_u, unigram_probabilities, no_of_sentences, no_of_words_each_sentence, use_unigram=True)

print("\n\n\nBIGRAM")
paragraph_bi = generate_paragraph(starting_words_b, bigram_probabilities, no_of_sentences, no_of_words_each_sentence, use_unigram=False)

type of unigram <class 'dict'>
type of bigram <class 'dict'>
starting_words_b: ['تھرڈ', 'ڈویزن', 'میں', 'پاس', 'ہونے', 'کی', 'وجہ', 'سے', 'یونیورسٹی', 'نے', 'ہم', 'کو', 'وظیفہ', 'دینا', 'مناسب', 'نہ', 'سمجھا']



UNIGRAM
Sentence No.1: <s> انعامات وقت بحث کے جی ہوسکا کرتے کا کی واقعہ نے آپ تجربے <\s>
Sentence No.2: <s> لفظی کی ہے پنپتا کا پہلے شیکسپیئر کے ضرور ہو نے کی کام کہا صورت تو کام <\s>
Sentence No.3: <s> دینے کے ضرور کا لیے یہ چلے کوشش دی جو کا ہم کمزور <\s>
Sentence No.4: <s> آئیں باریک مرزا سے تھا نصف کی جو دیا کے ہاتھ فہرست ہر کالج کوششوں نے لگ سوجھ <\s>
Sentence No.5: <s> ہے ایک الحال تازہ ہوئی نہیں ان قطعی اس چند جھٹ گے کون کراچی میرے ہم مشاغل اس <\s>
Sentence No.6: <s> سوز تو نظر ایسا میرے مشق ہم آگے کے <\s>
Sentence No.7: <s> بجے شروع کہ ہنرمندی صنعت ہماری نے <\s>
Sentence No.8: <s> ارادہ پندرہ جنہوں سے سوچنے مجال ہوگا ہوں <\s>
Sentence No.9: <s> رہے تھے میں ایک لوگ رکھتے سے کر اجی تمہاری کے میکبتھ ہماری کنکھیوں ہے نہیں صحرا پیدا <\s>
Sentence No.10: <s> بہت میں بہت <\s>

## Computing the Probability of Sentences

Compute the probability of each sentence that has been generated in the previous step. Refer to the lecture slides to see what does it mean to compute the _probability of a sentence_.

Apply the **unigram assumption** while computing these probabilities.


In [ ]:
sentence_probabilities = []
for sentence in paragraph_uni:
    sentence_probability = 1.0
    words = sentence.split()
    for word in words:
        word_probability = unigram_probabilities[word]
        sentence_probability *= word_probability

    sentence_probabilities.append(sentence_probability)
    print(f"Sentence: {sentence}, Probability: {sentence_probability}")


Sentence: <s> انعامات وقت بحث کے جی ہوسکا کرتے کا کی واقعہ نے آپ تجربے <\s>, Probability: 2.230994296118877e-40
Sentence: <s> لفظی کی ہے پنپتا کا پہلے شیکسپیئر کے ضرور ہو نے کی کام کہا صورت تو کام <\s>, Probability: 7.583387537308039e-48
Sentence: <s> دینے کے ضرور کا لیے یہ چلے کوشش دی جو کا ہم کمزور <\s>, Probability: 7.073722994989114e-39
Sentence: <s> آئیں باریک مرزا سے تھا نصف کی جو دیا کے ہاتھ فہرست ہر کالج کوششوں نے لگ سوجھ <\s>, Probability: 4.675836937513461e-57
Sentence: <s> ہے ایک الحال تازہ ہوئی نہیں ان قطعی اس چند جھٹ گے کون کراچی میرے ہم مشاغل اس <\s>, Probability: 2.6654599463578535e-55
Sentence: <s> سوز تو نظر ایسا میرے مشق ہم آگے کے <\s>, Probability: 4.485402492377973e-30
Sentence: <s> بجے شروع کہ ہنرمندی صنعت ہماری نے <\s>, Probability: 5.833481669528546e-24
Sentence: <s> ارادہ پندرہ جنہوں سے سوچنے مجال ہوگا ہوں <\s>, Probability: 2.8904136385457546e-30
Sentence: <s> رہے تھے میں ایک لوگ رکھتے سے کر اجی تمہاری کے میکبتھ ہماری کنکھیوں ہے نہیں صحرا پیدا <\s>, Probability

## Discussion and Evaluation

- Analyze the text generated, and mention 3 distinct observations. Also compare it with the input text and how different it is and why might that be.

- Do you notice any repetition of words in the generated sentences? If yes, how would you solve it?

- Is going upto `n=2` enough? What do you think would be a good value of n and why?


Answer here:
<div style="color:green;">
<b>Question#1</b>

- If I provide only two word as initials for my text generation (in bigram), that word occurs repeatedly in the generated text. This might be beacase while calculating the probability of the next word, the provided starting word lure the distribution towards itself making itself the most relevant word.

- Bigram output is making more sense than that of unigram. I think it's because bigram contains a very little context of the adjacent word while calculating the probability of the next word (which is not the case in unigram).

- In bigram, the conjunctions are the ones which are repeatedly appearing in a sentence. For example: اور and the words adjacent to اور are appearing more often.

- Probability of each sentence is very small, since I am generating very long sentences having unigrams (with each word having low individual pobability).

One reason the output text is different from the input text is the length of input sentence I am giving. The input sentence is smaller and hence have less words as context (especially in case of bigrams). Moreover, The generation process also involves randomness, especially when selecting the next word based on probabilities. As a result, the generated sentences can differ even when starting from the same input.

<b>Question#2</b>
Repeatition in more likely in bigram than in unigram (cz unigram includes more randomness). I am noticing some repeatition in my bigram generation which can avoided by including a condition that restricts the model from including the already generated random pair. Maybe use a language model like RNN, or Transformer instead of naive approach of n-grams. We can also normalize the generated output in post-processing,

<b>Question#3</b>
Going upto n=2 would make more logical sentences that'd make more sense grammtically but then there might be a chance of even more repeatition. We can try that out but I think n=2 would be fine for a small corpus that we have. If we had a large corpus then going n=3 might have resulted even better.
</div>
